In [1]:
# pyright: reportMissingImports=false
import neat
import numpy as np
#from tensorflow.keras.datasets import mnist
from keras.utils import plot_model


#(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
#train_images = train_images.reshape((len(train_images), 28*28)) / 255
#test_images = test_images.reshape((len(test_images), 28*28)) / 255
#train_labels = keras.utils.to_categorical(train_labels, 10)
#test_labels = keras.utils.to_categorical(test_labels, 10)

# load data from Drum Station 1.dt file
#train_images = np.loadtxt('A:/Профиль/Rab Table/Учёба/2/нейроэволюционные/Методические/data_to_use/cancer1.dt', delimiter=',', usecols=range(0, 9))
filename = 'A:/Профиль/Rab Table/Учёба/2/нейроэволюционные/Методические/data_to_use/cancer1 copy.dt'
raw_data = np.loadtxt(filename)

# what are modules to work with .dt files?
# https://stackoverflow.com/questions/17912307/u-ufeff-in-python-string

In [2]:
print(raw_data)

[[0.2 0.1 0.1 ... 0.1 1.  0. ]
 [0.2 0.1 0.1 ... 0.1 1.  0. ]
 [0.5 0.1 0.1 ... 0.1 1.  0. ]
 ...
 [0.6 1.  1.  ... 0.7 0.  1. ]
 [0.5 0.7 1.  ... 0.1 0.  1. ]
 [0.1 0.1 0.1 ... 0.1 1.  0. ]]


In [27]:
# make x and y from x
y = raw_data[:, 9] # [: , 9] - это значит, что мы берем все строки, но только столбец с индексом 9
x = raw_data[:, 0:9]
y = y.astype(float)

# make a list 'inputs' of sets from x like this: [(0.0, 0.0), (0.0, 1.0), (1.0, 0.0), (1.0, 1.0)]
inputs = []
for i in range(len(x)):
    inputs.append(tuple(x[i]))

# make a list 'outputs' of sets from y like this: [(0.0,), (1.0,), (1.0,), (0.0,)]
outputs = []
for i in range(len(y)):
    outputs.append(tuple([y[i]]))

print(outputs[0])
print(inputs[0])



(1.0,)
(0.2, 0.1, 0.1, 0.1, 0.2, 0.1, 0.2, 0.1, 0.1)


In [28]:
# split x and y into train, validation, test sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.75)

# convert y to one-hot
#y_train = keras.utils.to_categorical(y_train, 2)
#y_test = keras.utils.to_categorical(y_test, 2)

In [29]:
print(x_train[1])
print(y_train[1])
print(x_test[1])
print(y_test[1])

(0.1, 0.1, 0.1, 0.1, 0.2, 0.1, 0.2, 0.1, 0.2)
(1.0,)
(0.2, 0.1, 0.1, 0.1, 0.2, 0.1, 0.3, 0.1, 0.1)
(1.0,)


In [31]:
import pickle
import visualize
import graphviz

In [32]:
len(x_train)

174

In [37]:
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        fitness = len(x_train)
        for xi, xo in zip(x_train, y_train):
            output = net.activate(xi)
            output = tuple(output)
            try:
                fitness += (output[0] - xo[0]) ** 2
            except Exception as e:
                print(output)
                print(xo)
                print(xi)
                print(e)
                raise e
        genome.fitness = fitness

def main():
    config_path = 'A:/Профиль/Rab Table/Учёба/2/Neuroevolutionary-computing/Код/config-mnist'
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_path)

    population = neat.Population(config)
    winner = population.run(eval_genomes, n = 100)

    winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
    correct = 0
    for xi, xo in zip(x_test, y_test):
        output = winner_net.activate(xi)
        if np.argmax(output) == xo:
            correct += 1
    print('Accuracy:', correct / len(y_test))

    # Save the winner.
    with open('winner-mnist', 'wb') as f:
        pickle.dump(winner, f)
    
    import my_visualize as visualize


if __name__ == "__main__":
    main()

Accuracy: 0.35428571428571426
